In [1]:
import pandas as pd
staff =  pd.read_csv('staff.csv', index_col  =False)

In [2]:
staff

,Timestamp,name,email,blank,yes,lang,user,wiki
0,02/06/2015 06:15:40,Gilles,gilles@wikimedia.org,NaN,Yes.,French,Kouiskas,frwiki
1,02/06/2015 07:13:42,Stephane,sbisson@wikimedia.org,NaN,Yes.,French,stephanecbisson,frwiki
2,02/06/2015 07:45:19,Marcel,mforns@wikimedia.org,NaN,Yes.,Spanish,Marcelruizforns,eswiki
3,02/06/2015 08:12:52,Joseph,joal@wikimedia.org,NaN,Yes.,French,joal,frwiki
4,02/06/2015 08:57:25,Anne,agomez@wikimedia.org,NaN,Yes.,Spanish,NaN,eswiki
5,02/06/2015 10:05:29,Guillaume,gp@guillaumepaumier.com,NaN,Yes.,French,Guillom,frwiki
6,02/06/2015 11:48:56,Sylvia,sventura@wikimedia.org,NaN,Yes.,French,Slventura,frwiki
7,02/06/2015 17:22:31,Benoît,bevellin@wikimedia.org,NaN,Yes.,French,Trizek,frwiki
8,03/06/2015 01:24:50,Quim,qgil@wikimedia.org,NaN,Yes.,Spanish,QuimGil,eswiki
9,03/06/2015 11:28:10,Jorge,jvargas@wikimedia.org,NaN,Yes.,Spanish,Javp87,eswiki


In [3]:
staff_fr = staff[staff['wiki'] == 'frwiki']
staff_es = staff[staff['wiki'] == 'eswiki']

In [5]:
from ConfigParser import SafeConfigParser
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from util import get_parser
import json

config = '/home/ellery/wikimedia/missing_articles/missing_articles.ini'
cp = SafeConfigParser()
cp.read(config)
min_date = '20100600000000'
min_bytes = -1 #



def to_str(t):
    uid, contributions = t
    contributions = list(contributions)
    uname = contributions[0]['user']
    contributions = [{k: d[k] for k in ('language_code', 'id', 'page_title', 'num_edits', 'timestamp', 'bytes_added')} for d in contributions]
    contributions.sort(key = lambda x: x['timestamp'])
    if contributions[-1]['timestamp'] < min_date:
        return None
    else: 
        obj = {'uid': uid, 'uname': uname, 'contributions': contributions}
        return (uname, json.dumps(obj))

def get_contributions(cp, langs, user_set):
    contributions_file = os.path.join(cp.get('general', 'contributions_dir'), langs[0])
    contributions = sc.textFile(contributions_file)
    for lang in langs[1:]:
        contributions_file = os.path.join(cp.get('general', 'contributions_dir'), lang)
        contributions = contributions.union(sc.textFile(contributions_file))
    names = ["language_code", "user_id", "user", "id","page_title","num_edits","timestamp", "bytes_added"]

    contributions = contributions.map(get_parser(names))\
    .filter(lambda x: len(x) == 8)\
    .filter(lambda x: x['user'] in user_set)\
    .filter(lambda x: float(x['bytes_added']) > min_bytes)\
    .map(lambda x: (x['user'], x)).groupByKey()\
    .map(to_str)\
    .filter(lambda x: x is not None)\
    .collect()
    contributions = pd.DataFrame(contributions)
    contributions.columns =['user', 'history']
    return contributions

contributions_fr = get_contributions(cp, ['fr'], set(staff_fr['user']))

In [6]:
contributions_es = get_contributions(cp, ['es'], set(staff_es['user']))

In [7]:
d_es = contributions_es.merge(staff_es[['user', 'email']], on = 'user')

In [8]:
d_fr = contributions_fr.merge(staff_fr[['user', 'email']], on = 'user')

In [9]:
d_es.to_csv('/home/ellery/en-es/staff.tsv', encoding = 'utf8', sep = '\t')

In [10]:
d_fr.to_csv('/home/ellery/en-fr/staff.tsv', encoding = 'utf8', sep = '\t')

In [14]:
from pprint import pprint 
d_fr.index = d_fr['user']
pprint(d_fr.loc['Kouiskas']['history'])

'{"uname": "Kouiskas", "contributions": [{"page_title": "Bitcoin", "bytes_added": "0", "language_code": "fr", "num_edits": "1", "timestamp": "20131124093856", "id": "Q131723"}, {"page_title": "\\u00c9nur\\u00e9sie_nocturne", "bytes_added": "0", "language_code": "fr", "num_edits": "1", "timestamp": "20131129130521", "id": "Q318005"}], "uid": "Kouiskas"}'


In [13]:
d_fr

,user,history,email
user,,,
Slventura,Slventura,"{""uname"": ""Slventura"", ""contributions"": [{""pag...",sventura@wikimedia.org
Kouiskas,Kouiskas,"{""uname"": ""Kouiskas"", ""contributions"": [{""page...",gilles@wikimedia.org
Guillom,Guillom,"{""uname"": ""Guillom"", ""contributions"": [{""page_...",gp@guillaumepaumier.com
Trizek,Trizek,"{""uname"": ""Trizek"", ""contributions"": [{""page_t...",bevellin@wikimedia.org


In [15]:
d_es

,user,history,email
0,QuimGil,"{""uname"": ""QuimGil"", ""contributions"": [{""page_...",qgil@wikimedia.org
1,Javp87,"{""uname"": ""Javp87"", ""contributions"": [{""page_t...",jvargas@wikimedia.org
